In [2]:
import sys
sys.path.append('model')
sys.path.append('utils')
from utils_SH import *

# other modules
import numpy as np

from torch.autograd import Variable
from torchvision.utils import make_grid
import torch
import time

import cv2

import os

import matplotlib.pyplot as plt

def rgbshow(img_bgr):
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)


In [3]:
# Create normal for rendering half sphere
img_size = 256
x = np.linspace(-1, 1, img_size)
z = np.linspace(1, -1, img_size)
x, z = np.meshgrid(x, z)

mag = np.sqrt(x**2 + z**2)
valid = mag <=1
y = -np.sqrt(1 - (x*valid)**2 - (z*valid)**2)
x = x * valid
y = y * valid
z = z * valid
normal = np.concatenate((x[...,None], y[...,None], z[...,None]), axis=2)
normal = np.reshape(normal, (-1, 3))

In [4]:
# Create normal for rendering half sphere
img_size = 256
x = np.linspace(-1, 1, img_size)
z = np.linspace(1, -1, img_size)
x, z = np.meshgrid(x, z)

mag = np.sqrt(x**2 + z**2)
valid = mag <=1
y = -np.sqrt(1 - (x*valid)**2 - (z*valid)**2)
x = x * valid
y = y * valid
z = z * valid
normal = np.concatenate((x[...,None], y[...,None], z[...,None]), axis=2)
normal = np.reshape(normal, (-1, 3))

In [5]:
# Load Model
from defineHourglass_512_gray_skip import *

modelFolder = 'trained_model/'
my_network = HourglassNet()
my_network.load_state_dict(torch.load(os.path.join(modelFolder, 'trained_model_03.t7')))
my_network.cuda()
my_network.train(False)


HourglassNet(
  (pre_conv): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pre_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (light): lightingNet(
    (predict_FC1): Conv2d(27, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (predict_relu1): PReLU(num_parameters=1)
    (predict_FC2): Conv2d(128, 9, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (post_FC1): Conv2d(9, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (post_relu1): PReLU(num_parameters=1)
    (post_FC2): Conv2d(128, 27, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (post_relu2): ReLU()
  )
  (HG0): HourglassBlock(
    (upper): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (bn2): Instance

In [6]:
# Load input image 
img = cv2.imread('data/online_guy.jfif')
row, col, _ = img.shape
img = cv2.resize(img, (512, 512))
Lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
# rgbshow(img)

inputL = Lab[:, :, 0]
inputL = inputL.astype(np.float32)/255.0
inputL = inputL.transpose((0,1))
inputL = inputL[None, None, ...] #(1,1,512,512)
inputL = torch.from_numpy(inputL).cuda() #Variable(torch.from_numpy(inputL).cuda())


In [7]:
# Render Sphere
saveFolder = 'test_res'
lightFolder = 'data/example_light/'

sh = np.loadtxt(os.path.join(lightFolder, 'rotate_light_00.txt'.format(0)))[0:9] * 0.7
sh = np.squeeze(sh)

shading = get_shading(normal, sh)
value = np.percentile(shading, 95)
shading[shading > value] = value
shading = (shading-shading.min())/(shading.max()-shading.min())
shading = (shading*255.0).astype(np.uint8).reshape((256, 256)) * valid
cv2.imwrite(os.path.join(saveFolder, 'light_00.png'.format(0)), shading)


True

In [8]:
# Render Image using Network
sh = np.loadtxt(os.path.join(lightFolder, 'rotate_light_00.txt'.format(0)))[0:9] * 0.7
sh = sh.reshape(1, 9, 1, 1).astype(np.float32)
sh = torch.from_numpy(sh).cuda() #Variable(torch.from_numpy(sh).cuda())

outputImg, outputSH = my_network(inputL, sh, 0)
outputImg = outputImg[0].cpu().data.numpy().transpose((1,2,0)).squeeze()
outputImg = (outputImg*255.0).astype(np.uint8)
Lab[:,:,0] = outputImg
resultLab = cv2.cvtColor(Lab, cv2.COLOR_LAB2BGR)
resultLab = cv2.resize(resultLab, (col, row))
cv2.imwrite(os.path.join(saveFolder, 'online_guy_00.jpg'), resultLab)

#print(inputL.shape, sh.shape)
#print(my_network)

True

In [9]:
# Save and Load .pth
model_pth = os.path.join(saveFolder, 'model.pth')

# Save model to .pth
torch.save(my_network.state_dict(), model_pth)

# # Load model from .pth
# from defineHourglass_512_gray_skip import *
# my_network = HourglassNet()
# my_network.load_state_dict(torch.load(model_pth))
# my_network.cuda()
# my_network.train(False)


In [10]:
# Save to ONNX
#torch.Size([1, 1, 512, 512]) torch.Size([1, 9, 1, 1])

model_onnx = os.path.join(saveFolder, 'model.onnx')
dummy_input = (torch.randn(1,1,512,512).cuda(), torch.rand(1,9,1,1).cuda(), torch.tensor([0]))
torch.onnx.export(my_network, dummy_input, model_onnx, input_names=['x', 'target_light', 'skip_count'])

model\defineHourglass_512_gray_skip.py:49: TracerWarning: There are 2 live references to the data region being modified when tracing in-place operator add_. This might cause the trace to be incorrect, because all other views that also reference this data will not reflect this change in the trace! On the other hand, if all other views use the same memory chunk, but are disjoint (e.g. are outputs of torch.split), this might still be safe.
  out += x
model\defineHourglass_512_gray_skip.py:134: TracerWarning: There are 3 live references to the data region being modified when tracing in-place operator copy_ (possibly due to an assignment). This might cause the trace to be incorrect, because all other views that also reference this data will not reflect this change in the trace! On the other hand, if all other views use the same memory chunk, but are disjoint (e.g. are outputs of torch.split), this might still be safe.
  innerFeat[:,0:self.ncInput,:,:] = upFeat
model\defineHourglass_512_gray

In [1]:
import tensorflow

In [13]:
import onnx
from onnx2keras import onnx_to_keras

# # Load ONNX model
o_model = onnx.load(model_onnx)

# # Call the converter (input - is the main model input name, can be different for your model)
k_model = onnx_to_keras(o_model, ['input'])

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> x.
DEBUG:onnx2keras:Input 1 -> target_light.
DEBUG:onnx2keras:Input 2 -> skip_count.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> 372.
DEBUG:onnx2keras:Output 1 -> 202.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight HG0.low1.bn1.bias with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.num_batches_tracked with shape ().
DEBUG:onnx2keras:Found weight HG0.low1.bn1.running_mean with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.running_var with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.weight with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn2.bias with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn2.num_batches_tracked with shape ().
DEBUG:onnx2keras:Found weight HG0.low1.bn2.running_mean with shape (155,).
DEBUG:onnx2keras:Found weight 

DEBUG:onnx2keras:Found weight pre_bn.running_mean with shape (16,).
DEBUG:onnx2keras:Found weight pre_bn.running_var with shape (16,).
DEBUG:onnx2keras:Found weight pre_bn.weight with shape (16,).
DEBUG:onnx2keras:Found weight pre_conv.bias with shape (16,).
DEBUG:onnx2keras:Found weight pre_conv.weight with shape (16, 1, 5, 5).
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 115
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [5, 5], 'pads': [2, 2, 2, 2], 'strides': [1, 1], 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name x).
DEBUG:onnx2keras:The input not found in layers / model inputs.


AttributeError: Current node is not in weights / model inputs / layers.

In [26]:
from pytorch2keras import pytorch_to_keras
from torch.autograd import Variable


dummy_input = [torch.randn(1,1,512,512).cuda(), torch.rand(1,9,1,1).cuda(), torch.tensor([0])]
# dummy_input = Variable(dummy_input)
# args = [dummy_input]
# print(args)
k_model = pytorch_to_keras(my_network, args=dummy_input)
# my_network(*dummy_input)

#input_shapes=None, change_ordering=False, verbose=False, name_policy=None

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0', 'input_1', 'input_2']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0', 'output_1']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:Input 1 -> input_1.
DEBUG:onnx2keras:Input 2 -> input_2.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Output 1 -> output_1.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight HG0.low1.bn1.bias with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.num_batches_tracked with shape ().
DEBUG:onnx2keras:Found weight HG0.low1.bn1.running_mean with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.running_var with shape (155,).
DEBUG:onnx2keras:Found weight HG0.low1.bn1.weight with shape (155,).
DEBUG:onnx2keras:Found weight HG0.lo

DEBUG:onnx2keras:Found weight output.bias with shape (1,).
DEBUG:onnx2keras:Found weight output.weight with shape (1, 16, 1, 1).
DEBUG:onnx2keras:Found weight pre_bn.bias with shape (16,).
DEBUG:onnx2keras:Found weight pre_bn.num_batches_tracked with shape ().
DEBUG:onnx2keras:Found weight pre_bn.running_mean with shape (16,).
DEBUG:onnx2keras:Found weight pre_bn.running_var with shape (16,).
DEBUG:onnx2keras:Found weight pre_bn.weight with shape (16,).
DEBUG:onnx2keras:Found weight pre_conv.bias with shape (16,).
DEBUG:onnx2keras:Found weight pre_conv.weight with shape (16, 1, 5, 5).
DEBUG:onnx2keras:Found input input_0 with shape [1, 512, 512]
DEBUG:onnx2keras:Found input input_1 with shape [9, 1, 1]
DEBUG:onnx2keras:Found input input_2 with shape []
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Conv
DEBUG:onnx2keras:node_name: 115
DEBUG:onnx2keras:node_params: {'dilations': [1, 1], 'group': 1, 'kernel_shape': [5, 5], '

AttributeError: module 'tensorflow' has no attribute 'contrib'

In [28]:
tensorflow.__version__

'2.0.0'